In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [65]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        #self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        #self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3)
        #self.conv5 = nn.Conv2d(256, 512, 3)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        self.fc1 = nn.Linear(3*3*32, 128)
        self.out = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.bn1(self.conv2(x))))
        x = F.relu(self.conv3(x))
        x = x.reshape (-1, 3*3*32)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.out(x)
        return x
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        #x = x.view(-1, 10)
        #return F.log_softmax(x)

In [66]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
print(use_cuda)
#print(torch.cuda.device_count())
#print(torch.cuda.memory_allocated(0)/1024)
#use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
#device = torch.device("cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
         MaxPool2d-2            [-1, 8, 13, 13]               0
            Conv2d-3           [-1, 16, 11, 11]           1,168
       BatchNorm2d-4           [-1, 16, 11, 11]              32
         MaxPool2d-5             [-1, 16, 5, 5]               0
            Conv2d-6             [-1, 32, 3, 3]           4,640
            Linear-7                  [-1, 128]          36,992
            Linear-8                   [-1, 10]           1,290
Total params: 44,202
Trainable params: 44,202
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 0.17
Estimated Total Size (MB): 0.26
----------------------------------------------------------------


In [67]:


torch.manual_seed(1)
batch_size_train = 64
batch_size_test = 1000

kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size_train, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size_test, shuffle=True, **kwargs)


In [68]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [69]:

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.009)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.11912451684474945 batch_id=937: 100%|██████████| 938/938 [00:14<00:00, 63.94it/s]  



Test set: Average loss: -10.6847, Accuracy: 9791/10000 (97.91%)



loss=0.0721498653292656 batch_id=937: 100%|██████████| 938/938 [00:11<00:00, 82.73it/s]    



Test set: Average loss: -10.6199, Accuracy: 9842/10000 (98.42%)



loss=0.0009579280158504844 batch_id=937: 100%|██████████| 938/938 [00:11<00:00, 83.24it/s]  


...


Test set: Average loss: -21.8474, Accuracy: 9861/10000 (98.61%)



loss=2.4058963390416466e-05 batch_id=937: 100%|██████████| 938/938 [00:12<00:00, 77.90it/s] 



Test set: Average loss: -26.6087, Accuracy: 9869/10000 (98.69%)



loss=7.58784226491116e-05 batch_id=937: 100%|██████████| 938/938 [00:11<00:00, 84.15it/s]   



Test set: Average loss: -28.6080, Accuracy: 9870/10000 (98.70%)

